<a href="https://colab.research.google.com/github/DebStar17/2-Piece-Chess/blob/main/2PC_v2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# P1 (Knight) and P2 (King) Movements

In [ ]:
#Define p1 and p2 movements
movement_p1 = [[2, -1], [2, 1], [1, -2], [1, 2], [-2, -1], [-2, 1], [-1, -2], [-1, 2]]
movement_p2 = [[0, 1], [0, -1], [1, 0], [-1, 0], [-1, -1], [-1, 1], [1, 1], [1, -1]]

# All functions

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap
import random
import pickle

### Show Grid

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def show_grid(grid):
  grid = np.array(grid)
  rows, cols = grid.shape

  # 1. Define the color map as an explicit dictionary (Value: RGB Tuple)
  # The color values are given as (R, G, B) tuples, normalized to 0-1.
  color_map = {
      0: (1.0, 1.0, 1.0), # White
      1: (1.0, 0.0, 0.0), # Red
      2: (0.0, 0.0, 1.0), # Blue
      3: (0.0, 0.0, 0.0), # Black
      4: (1.0, 1.0, 0.8)  # Yellow
  }

  # 2. Create the 3D RGB array
  # We initialize an empty array of shape (rows, cols, 3) with float type
  rgb_grid = np.zeros((rows, cols, 3), dtype=float)

  # Iterate through the grid and assign RGB colors
  for i in range(rows):
    for j in range(cols):
      value = grid[i, j]
      # Use .get() for safety, defaulting to white if an unexpected value appears
      rgb_grid[i, j] = color_map.get(value, (1.0, 1.0, 1.0))

  # 3. Display the RGB array directly
  plt.imshow(rgb_grid, interpolation='nearest')
  plt.gca().invert_yaxis()
  #

  # --- Text Labeling and Gridlines (Same as before) ---
  for i in range(rows):
    for j in range(cols):
      cell_value = grid[i, j]
      text_to_display = ''
      text_color = 'black'

      if cell_value == 1:
        text_to_display = 'N'
        text_color = 'white'
      elif cell_value == 2:
        text_to_display = 'K'
        text_color = 'white'
      else:
        continue

      plt.text(j, i, text_to_display,
               ha='center', va='center',
               color=text_color, fontsize=10,
               fontweight='bold')

  # Setting minor tick locations and drawing gridlines
  plt.gca().set_xticks(np.arange(-0.5, cols, 1), minor=True)
  plt.gca().set_yticks(np.arange(-0.5, rows, 1), minor=True)
  plt.grid(which='minor', color='black', linestyle='-', linewidth=2)
  plt.show()

### Surrounding Cells

In [ ]:
def surr(p1x, p1y, p1, grid):
  gr = len(grid)
  surrli = []
  for x, y in p1:
    if -1 < p1x + x < gr and -1 < p1y + y < gr and grid[p1y + y][p1x + x] != 3:
      surrli.append((p1x + x, p1y + y))
  return surrli

### Play function

In [ ]:
import random
def random_play(my_p, opp, grid, my_surr, all_games, player="p1"):
  if opp in my_surr:
    return opp #"my_p wins"

  return random.choice(my_surr)


In [ ]:
import random
def trained_play(my_p, opp, grid, my_surr, all_games, player="p1"): # If training dataset exists - wherein, all_games is a list of lists
  if opp in my_surr:
    return opp #"my_p wins"

  if not all_games:
    return random.choice(my_surr)

  next_moves = []
  for *moves, win in all_games:
    if player == "p1":
      # Ensure i+1 does not go out of bounds for moves list
      for i in range(0, len(moves) - 1, 2):
        if moves[i] == (my_p, opp) and (win[0] == 1 or win[0] == 2):
          next_moves.append(moves[i+1][0])

    if player == "p2":
      # Ensure i+1 does not go out of bounds for moves list
      for i in range(1, len(moves) - 1, 2):
        if moves[i] == (opp, my_p) and (win[1] == 1 or win[1] == 2):
          next_moves.append(moves[i+1][1])

  print(next_moves)
  if next_moves:
    return random.choice(next_moves)
  else:
    return random.choice(my_surr)

### Gameplay

In [ ]:
def train(grid, initial_p1, initial_p2, filename = "data.dat"):
  p1 = initial_p1
  p2 = initial_p2

  try:
    with open(filename, "rb") as f:
      all_games = pickle.load(f)
  except:
    all_games = []

  curr_game = [(p1,p2)]

  import copy
  grid2 = copy.deepcopy(grid)
  w = 0

  for play in range(50):
    # Modifies grid in-place
    if play%2 == 0:
      my_surr = surr(p1[0], p1[1], movement_p1, grid2)
      grid2[p1[1]][p1[0]] = 0

      p1 = random_play(p1, p2, grid2, my_surr, all_games, player="p1")
      grid2[p1[1]][p1[0]] = 1

    else:
      my_surr = surr(p2[0], p2[1], movement_p2, grid2)
      grid2[p2[1]][p2[0]] = 0

      p2 = random_play(p2, p1, grid2, my_surr, all_games, player="p2")
      grid2[p2[1]][p2[0]] = 2

    curr_game.append((p1,p2))
    if p1 == p2:
      if play%2==0:
        curr_game.append((1,0))
        w = 1
        break
      else:
        curr_game.append((0,1))
        w = 1
        break

  if not w:
    curr_game.append((2,2))

  all_games.append(curr_game)

  with open(filename, "wb") as f:
    pickle.dump(all_games, f)

In [ ]:
def showdown(grid, initial_p1, initial_p2, filename = "data.dat"):
  p1 = initial_p1
  p2 = initial_p2
  try:
    with open(filename, "rb") as f:
      all_games = pickle.load(f)
  except:
    all_games = []

  curr_game = [(p1,p2)]

  import copy
  grid2 = copy.deepcopy(grid)
  w = 0

  for play in range(50):
    print(play)

    # Modifies grid in-place
    if play%2 == 0:
      my_surr = surr(p1[0], p1[1], movement_p1, grid2)

      for x,y in my_surr:
        if not (1 <= grid2[y][x] <= 2):
          grid2[y][x] = 4
      show_grid(grid2)

      for x,y in my_surr:
        if not (1 <= grid2[y][x] <= 2):
          grid2[y][x] = 0

      grid2[p1[1]][p1[0]] = 0
      p1 = trained_play(p1, p2, grid2, my_surr, all_games, player="p1")
      grid2[p1[1]][p1[0]] = 1

    else:
      my_surr = surr(p2[0], p2[1], movement_p2, grid2)

      for x,y in my_surr:
        if not (1 <= grid2[y][x] <= 2):
          grid2[y][x] = 4
      show_grid(grid2)

      for x,y in my_surr:
        if not (1 <= grid2[y][x] <= 2):
          grid2[y][x] = 0

      grid2[p2[1]][p2[0]] = 0
      p2 = trained_play(p2, p1, grid2, my_surr, all_games, player="p2")
      grid2[p2[1]][p2[0]] = 2

    if input("Press Enter to play: ") == "stop":
      break

    curr_game.append((p1,p2))
    if p1 == p2:
      if play%2==0:
        # P1 wins
        print(play+1)
        show_grid(grid2)
        print("Knight wins")
        curr_game.append((1,0))
        w = 1
        break
      else:
        print(play+1)
        show_grid(grid2)
        print("King wins")
        curr_game.append((0,1))
        w = 1
        break

  if not w:
    curr_game.append((2,2))

  all_games.append(curr_game)

  with open(filename, "wb") as f:
    pickle.dump(all_games, f)

In [ ]:
def user_play(grid, initial_p1, initial_p2, filename = "data.dat", user = "P1"):
  p1 = initial_p1
  p2 = initial_p2
  try:
    with open(filename, "rb") as f:
      all_games = pickle.load(f)
  except:
    all_games = []

  curr_game = [(p1,p2)]

  import copy
  grid2 = copy.deepcopy(grid)
  w = 0

  for play in range(50):
    print(play)

    # Modifies grid in-place
    if play%2 == 0:
      my_surr = surr(p1[0], p1[1], movement_p1, grid2)

      for x,y in my_surr:
        if not (1 <= grid2[y][x] <= 2):
          grid2[y][x] = 4

      show_grid(grid2)

      for x,y in my_surr:
        if not (1 <= grid2[y][x] <= 2):
          grid2[y][x] = 0

      grid2[p1[1]][p1[0]] = 0

      if user == "P1":
        while True:
          e = tuple(map(int, input("Where to move Knight? ").split()))
          if e in my_surr:
            p1 = e
            break
          else:
            print("Invalid Move")
            continue
      else:
        p1 = trained_play(p1, p2, grid2, my_surr, all_games, player="p1")
        if input("Press Enter to play: ") == "stop":
          break
      grid2[p1[1]][p1[0]] = 1

    else:
      my_surr = surr(p2[0], p2[1], movement_p2, grid2)

      for x,y in my_surr:
        if not (1 <= grid2[y][x] <= 2):
          grid2[y][x] = 4
      show_grid(grid2)

      for x,y in my_surr:
        if not (1 <= grid2[y][x] <= 2):
          grid2[y][x] = 0

      for x,y in my_surr:
        grid2[y][x] = 0

      grid2[p2[1]][p2[0]] = 0

      if user == "P2":
        while True:
          e = tuple(map(int, input("Where to move King? ").split()))
          if e in my_surr:
            p2 = e
            break
          else:
            print("Invalid Move")
            continue
      else:
        p2 = trained_play(p2, p1, grid2, my_surr, all_games, player="p2")
        if input("Press Enter to play: ") == "stop":
          break

      grid2[p2[1]][p2[0]] = 2



    curr_game.append((p1,p2))
    if p1 == p2:
      if play%2==0:
        # P1 wins
        print(play+1)
        show_grid(grid2)
        print("Knight wins")
        curr_game.append((1,0))
        w = 1
        break
      else:
        print(play+1)
        show_grid(grid2)
        print("King wins")
        curr_game.append((0,1))
        w = 1
        break

  if not w:
    curr_game.append((2,2))

  all_games.append(curr_game)

  with open(filename, "wb") as f:
    pickle.dump(all_games, f)

### Main

In [ ]:
def Make_Grid(): # Takes input --> returns grid (no players)
  # Create data.dat
  f = open("data.dat", "wb")
  f.close()

  # Make Grid
  gr = int(input("Enter the size of the grid: "))
  bias = list(map(int, input("Enter Bias for space and wall: ").split()))
  grid = [[0 for _ in range(gr)] for _ in range(gr)]

  # Adding Walls
  for i in grid:
    for j in range(len(i)):
      i[j] = random.choices([0, 3], weights=bias, k=1)[0]

  show_grid(grid)
  return grid

In [ ]:
def Train_Bots(grid): # Puts in players, starts Training - returns grid with players, p1, p2
  import copy
  grid = copy.deepcopy(grid)
  import time
  time.sleep(0.1)
  p1x, p1y = map(int, input("Enter pos of Knight: ").split())
  p2x, p2y = map(int, input("Enter pos of King..: ").split())

  grid[p1y][p1x] = 1
  grid[p2y][p2x] = 2

  show_grid(grid)

  time.sleep(0.1)
  thresh = int(input("Enter Training Threshold: "))
  for i in range(thresh):
    train(grid, (p1x, p1y), (p2x,p2y), filename = "data.dat")

  print("\nTRAINING COMPLETE\n")
  return grid, (p1x, p1y), (p2x,p2y)

In [ ]:
def Bot_Bot(grid, p1, p2):
  print("---GAMEPLAY---")
  showdown(grid, p1, p2, filename = "data.dat")

In [ ]:
def You_Bot(grid, p1, p2):
  p = int(input("""Select Player:
  1. Play as Knight (P1)
  2. Play as King   (P2)
"""))
  if (1<=p<=2):
    print("---GAMEPLAY---")
    user_play(grid, p1, p2, filename = "data.dat", user = f"P{p}")
  else:
    print("Invalid Input")

# User Interface

In [ ]:
grid = Make_Grid() # Make the Grid/Chess Board

In [ ]:
grid1, p1, p2 = Train_Bots(grid) # Train the bots

In [ ]:
Bot_Bot(grid1, p1, p2) # Bot v/s Bot

In [ ]:
You_Bot(grid1, p1, p2) # You v/s Bot